<a href="https://colab.research.google.com/github/vervainalthor/Data-analytics-for-medium/blob/main/BASIC_RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traditional Customer Segmentation Using RFM Analysis 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.mixture import GaussianMixture

In [ ]:
dfacct = pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/account.csv', sep=";", header=0)

dfcard = pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/card.csv', sep=";", header=0)
dfclient = pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/client.csv', sep=";", header=0)

dfdistrict = pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/district.csv', sep=";", header=0)

dfdisp = pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/disp.csv', sep=";", header=0)

dftrans=pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/trans.csv', sep=";",
                    header=0)
dfloan=pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/loan.csv', sep=";",
                    header=0)
dforder=pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/order.csv', sep=";",
                    header=0)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


#Renaming columns and replacing Czech categories with an english equivalent

## Account table

In [ ]:
dfacct['date']= pd.to_datetime(dfacct['date'], format='%y%m%d')

In [ ]:
dfacct.loc[dfacct['frequency'].isin(['POPLATEK MESICNE']), 'frequency'] = 'monthly issuance'

In [ ]:
dfacct.loc[dfacct['frequency'].isin(['POPLATEK TYDNE']), 'frequency'] = 'weekly issuance'

In [ ]:
dfacct.loc[dfacct['frequency'].isin(['POPLATEK PO OBRATU']), 'frequency'] = 'post tranissuance'

In [ ]:
dfacct[dfacct['frequency'].str.contains('post')]

,account_id,district_id,frequency,date
6,1539,1,post tranissuance,1993-01-03
14,485,6,post tranissuance,1993-01-04
130,1965,57,post tranissuance,1993-02-08
148,539,13,post tranissuance,1993-02-16
175,1050,57,post tranissuance,1993-02-24
...,...,...,...,...
4269,3103,1,post tranissuance,1997-09-21
4397,2211,61,post tranissuance,1997-11-10
4409,3156,1,post tranissuance,1997-11-16
4425,1523,52,post tranissuance,1997-11-24


In [ ]:
dfacct.head()

,account_id,district_id,frequency,date
0,576,55,monthly issuance,1993-01-01
1,3818,74,monthly issuance,1993-01-01
2,704,55,monthly issuance,1993-01-01
3,2378,16,monthly issuance,1993-01-01
4,2632,24,monthly issuance,1993-01-02


In [ ]:
dfdisp= dfdisp[dfdisp["type"].str.contains("OWNER")]

In [ ]:
dfdisp.head()

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
3,4,4,3,OWNER
5,6,6,4,OWNER
6,7,7,5,OWNER


##Card ownership

In [ ]:
dfcard['issued']= pd.to_datetime(dfcard['issued'], format='%y%m%d %H:%M:%S') 

In [ ]:
# Extract only the date component
dfcard['issued'] = dfcard['issued'].dt.date

In [ ]:
dfcard.head()

,card_id,disp_id,type,issued
0,1005,9285,classic,1993-11-07
1,104,588,classic,1994-01-19
2,747,4915,classic,1994-02-05
3,70,439,classic,1994-02-08
4,577,3687,classic,1994-02-15


## Demograhic key details

In [ ]:
dfdistrict = dfdistrict.rename(columns={'A1': 'district_code', 'A2': 'district_name', 'A3': 'region',
                                        'A4': 'inhabitant_count', 'A8': 'inhabitants<10kcount', 'A12': 'unemployment rate_95',
                                        'A5': 'inhabitants<499count', 'A9.': 'citiescount', 'A13': 'unemployment rate_96',
                                        'A6': 'inhabitants>5h<2kcount', 'A10': 'urban_inhabitantratio', 'A14': 'entrepreneurs_per_1000','A15': 'committedcrimes_95',
                                        'A7': 'inhabitants<2k<10kcount', 'A11': 'average_salary', 'A16': 'committedcrimes_95'})


# CREATING THE RFM TABLE WITH TRANSACTION TABLE

* renaming content

In [ ]:
dftrans['date']= pd.to_datetime(dftrans['date'], format='%y%m%d') 

In [ ]:
dftrans.loc[dftrans['type'].isin(['PRIJEM']), 'type'] = 'credit'

In [ ]:
dftrans.loc[dftrans['type'].isin(['VYDAJ']), 'type'] = 'debit'

In [ ]:
dftrans['operation'] = dftrans['operation'].replace({'VYBER KARTOU': 'creditcardwitdrawal','VKLAD':'credit_in_cash', "PREVOD Z UCTU":'anotherbankcredit'
,"VYBER":'cashdebit',"PREVOD NA UCET":'tranferdebit'})

In [ ]:
dftrans['operation'].unique()

array(['credit_in_cash', 'anotherbankcredit', 'cashdebit', nan,
       'tranferdebit', 'creditcardwitdrawal'], dtype=object)

In [ ]:
dftrans.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,1993-01-01,credit,credit_in_cash,700.0,700.0,NaN,NaN,NaN
1,171812,576,1993-01-01,credit,credit_in_cash,900.0,900.0,NaN,NaN,NaN
2,207264,704,1993-01-01,credit,credit_in_cash,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,1993-01-01,credit,credit_in_cash,600.0,600.0,NaN,NaN,NaN
4,579373,1972,1993-01-02,credit,credit_in_cash,400.0,400.0,NaN,NaN,NaN


#      RFM ANALYSIS DATA GATHERING
* RFM for traditional segmentation 
* RFM for the purpose of churn prediction 
* RFM for customer lifetime value Predictions

In [ ]:
dftrans['date'].max()

Timestamp('1998-12-31 00:00:00')

In [ ]:
dftrans['date'].min()

Timestamp('1993-01-01 00:00:00')

## Overall Recency 

In [ ]:
dftransR = dftrans.copy()
#getting just the dates we need for
dftransR.drop(['trans_id', 'type',	'operation','amount','balance','k_symbol'	,'bank'	,'account'], axis=1, inplace=True) 

dftransR['last_date'] = dftrans.groupby('account_id')['date'].transform('max') #grouping my max date
dftransR = dftransR[dftrans['date'] == dftransR['last_date']] #removing unncessary rows
dftransR.drop(['last_date'], axis=1, inplace=True) 
dftransR['Time Difference'] = (pd.Timestamp('1999-01-01')- dftrans['date'] ).dt.days #time
dftransR.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-23-169e51b22a8d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftransR['Time Difference'] = (pd.Timestamp('1999-01-01')- dftrans['date'] ).dt.days #time


,account_id,date,Time Difference
369246,799,1996-08-25,859
378587,1720,1996-09-07,846
433290,2029,1996-12-09,753
501925,459,1997-03-01,671
507351,2180,1997-03-10,662


In [ ]:
dftransR['Time Difference'] += np.random.normal(0, 0.0001, size=len(dftransR)) #I get on getting errors due to duplicates so i tried to intoduce noise
percentiles = np.percentile(dftransR['Time Difference'], [0, 33.33, 66.67, 100])  #the into thirds
bins = [percentiles[0], percentiles[1], percentiles[2], percentiles[3]]  # bin creation
dftransR['Rvalue'] = pd.cut(dftransR['Time Difference'], bins=bins, labels=[3, 2, 1],include_lowest=True, right=True)  # it's 3,2,1 because a high time difference is bad
dftransR['definition R'] = pd.cut(dftransR['Time Difference'], bins=bins, labels=['Recent', 'Reactivation effort', 'churned'],include_lowest=True, right=True) 
dftransT = dftransR.copy() #I need this for active period later
dftransR.head()

<ipython-input-24-a93a45c792b4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftransR['Time Difference'] += np.random.normal(0, 0.0001, size=len(dftransR)) #I get on getting errors due to duplicates so i tried to intoduce noise
<ipython-input-24-a93a45c792b4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftransR['Rvalue'] = pd.cut(dftransR['Time Difference'], bins=bins, labels=[3, 2, 1],include_lowest=True, right=True)  # it's 3,2,1 because a high time difference is bad
<ipython-input-24-a93a45c792b

,account_id,date,Time Difference,Rvalue,definition R
369246,799,1996-08-25,859.000086,1,churned
378587,1720,1996-09-07,845.999934,1,churned
433290,2029,1996-12-09,752.999917,1,churned
501925,459,1997-03-01,671.000004,1,churned
507351,2180,1997-03-10,662.000003,1,churned


In [ ]:
dftransR.drop(['date','Time Difference'], axis=1, inplace=True)
dftransR.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,account_id,Rvalue,definition R
369246,799,1,churned
378587,1720,1,churned
433290,2029,1,churned
501925,459,1,churned
507351,2180,1,churned


## FREQUENCY

## Determining a Average Frequency

In [ ]:
dftransF = dftrans.copy() 

dftransF= dftransF.groupby('account_id').size().reset_index(name='Frequency')
dftransF['Frequency'] = dftransF['Frequency'].astype(float)

dftransF.head()


,account_id,Frequency
0,1,239.0
1,2,478.0
2,3,117.0
3,4,186.0
4,5,84.0


### Getting the weights which is the lifetime of the customer

In [ ]:
overall_weight= pd.merge(dfacct.copy(),dftransT, on='account_id')

overall_weight['Active Period'] = round((overall_weight['date_y'] - overall_weight['date_x'].fillna(pd.Timestamp('1999-01-01'))).dt.days/365,1) #Active lifetime
overall_weight['lifetime'] = round(((pd.Timestamp('1999-01-01')- overall_weight['date_x'].fillna(overall_weight['date_x']) ).dt.days)/365,1)   #lifetime

overall_weight.drop(['district_id',	'frequency',	'date_y','date_x','Time Difference',	'Rvalue',	'definition R'], axis=1, inplace=True)

overall_weight.head()

,account_id,Active Period,lifetime
0,576,6.0,6.0
1,3818,6.0,6.0
2,704,6.0,6.0
3,2378,6.0,6.0
4,2632,6.0,6.0


drop and merge

###Creating the weights

# M- MONETARY VALUE LET'S USE TRANSACTION VOLUME OVER THE PERIOD

In [ ]:
dftransM = dftrans.groupby('account_id'	).sum()
dftransM.drop(['trans_id', 'balance','account'], axis=1, inplace=True)

dftransM.reset_index(inplace=True)
dftransM.head()


,account_id,amount
0,1,375174.5
1,2,3151479.3
2,3,295021.8
3,4,350971.4
4,5,166881.4


In [ ]:
df_RFM = pd.merge(pd.merge(pd.merge(dftransR,dftransF, on='account_id'), dftransM, on='account_id'),overall_weight,on='account_id')

df_RFM['Avg_frequency'] = df_RFM['Frequency']/df_RFM['Active Period']
df_RFM['Avg_amount'] = df_RFM['amount']/df_RFM['Active Period']

df_RFM.head()

,account_id,Rvalue,definition R,Frequency,amount,Active Period,lifetime,Avg_frequency,Avg_amount
0,799,1,churned,39.0,71548.7,2.9,5.2,13.448276,24671.965517
1,1720,1,churned,16.0,33400.0,1.3,3.6,12.307692,25692.307692
2,2029,1,churned,36.0,69100.0,3.5,5.6,10.285714,19742.857143
3,459,1,churned,49.0,83500.0,4.1,5.9,11.951220,20365.853659
4,2180,1,churned,41.0,78500.0,3.4,5.2,12.058824,23088.235294


In [ ]:
percentilesF = np.percentile(df_RFM['Avg_frequency'], [0, 33.33, 66.67, 100])  #the into thirds
binsF = [percentilesF[0], percentilesF[1], percentilesF[2], percentilesF[3]]  # bin creation
df_RFM['Fvalue'] = pd.cut(df_RFM['Avg_frequency'], bins=binsF, labels=[1, 2, 3],include_lowest=True, right=True)#,include_lowest=True, right=False)  # it's 3,2,1 because a high time difference is bad
df_RFM['definition F'] = pd.cut(df_RFM['Avg_frequency'], bins=binsF, labels=['low activity', 'Mid-Activity', ' Very Active'],include_lowest=True, right=True) 
df_RFM.head()


,account_id,Rvalue,definition R,Frequency,amount,Active Period,lifetime,Avg_frequency,Avg_amount,Fvalue,definition F
0,799,1,churned,39.0,71548.7,2.9,5.2,13.448276,24671.965517,1,low activity
1,1720,1,churned,16.0,33400.0,1.3,3.6,12.307692,25692.307692,1,low activity
2,2029,1,churned,36.0,69100.0,3.5,5.6,10.285714,19742.857143,1,low activity
3,459,1,churned,49.0,83500.0,4.1,5.9,11.951220,20365.853659,1,low activity
4,2180,1,churned,41.0,78500.0,3.4,5.2,12.058824,23088.235294,1,low activity


In [ ]:
percentilesM = np.percentile(df_RFM['Avg_amount'], [0, 33.33, 66.67, 100])  #the into thirds
binsM = [percentilesM[0], percentilesM[1], percentilesM[2], percentilesM[3]]  # bin creation
df_RFM['Mvalue'] = pd.cut(df_RFM['Avg_amount'], bins=binsM, labels=[1, 2, 3],include_lowest=True,  right=True)  
df_RFM['definition M'] = pd.cut(df_RFM['Avg_amount'], bins=binsM, labels=['low value', 'Mid value', 'high value'],include_lowest=True, right=True) 
df_RFM.head()



,account_id,Rvalue,definition R,Frequency,amount,Active Period,lifetime,Avg_frequency,Avg_amount,Fvalue,definition F,Mvalue,definition M
0,799,1,churned,39.0,71548.7,2.9,5.2,13.448276,24671.965517,1,low activity,1,low value
1,1720,1,churned,16.0,33400.0,1.3,3.6,12.307692,25692.307692,1,low activity,1,low value
2,2029,1,churned,36.0,69100.0,3.5,5.6,10.285714,19742.857143,1,low activity,1,low value
3,459,1,churned,49.0,83500.0,4.1,5.9,11.951220,20365.853659,1,low activity,1,low value
4,2180,1,churned,41.0,78500.0,3.4,5.2,12.058824,23088.235294,1,low activity,1,low value


In [ ]:
percentilesL = np.percentile(df_RFM['lifetime'], [0, 33.33, 66.67, 100])  #the into thirds
binsL = [percentilesL[0], percentilesL[1], percentilesL[2], percentilesL[3]]  # bin creation
df_RFM['Lvalue'] = pd.cut(df_RFM['lifetime'], bins=binsL, labels=[1, 2, 3],include_lowest=True, right=True)
df_RFM['definition L'] = pd.cut(df_RFM['lifetime'], bins=binsL, labels=['New Customer', 'Mid-life Customer', 'Long time customer'],include_lowest=True, right=True)    
df_RFM.head()

,account_id,Rvalue,definition R,Frequency,amount,Active Period,lifetime,Avg_frequency,Avg_amount,Fvalue,definition F,Mvalue,definition M,Lvalue,definition L
0,799,1,churned,39.0,71548.7,2.9,5.2,13.448276,24671.965517,1,low activity,1,low value,3,Long time customer
1,1720,1,churned,16.0,33400.0,1.3,3.6,12.307692,25692.307692,1,low activity,1,low value,2,Mid-life Customer
2,2029,1,churned,36.0,69100.0,3.5,5.6,10.285714,19742.857143,1,low activity,1,low value,3,Long time customer
3,459,1,churned,49.0,83500.0,4.1,5.9,11.951220,20365.853659,1,low activity,1,low value,3,Long time customer
4,2180,1,churned,41.0,78500.0,3.4,5.2,12.058824,23088.235294,1,low activity,1,low value,3,Long time customer


In [ ]:
df_RFM['Avg_amount'].min()

16875.0

In [ ]:
nan_rows = df_RFM[df_RFM['Rvalue'].isnull()]
nan_rows.head(15)

,account_id,Rvalue,definition R,Frequency,amount,Active Period,lifetime,Avg_frequency,Avg_amount,Fvalue,definition F,Mvalue,definition M,Lvalue,definition L


In [ ]:
nan_rows = df_RFM[df_RFM['definition L'].isnull()]
nan_rows.head(15)

,account_id,Rvalue,definition R,Frequency,amount,Active Period,lifetime,Avg_frequency,Avg_amount,Fvalue,definition F,Mvalue,definition M,Lvalue,definition L


In [ ]:

df_RFM["RFML"] = df_RFM['Rvalue'].astype(str) +df_RFM['Fvalue'].astype(str) + df_RFM['Mvalue'].astype(str)+df_RFM['Lvalue'].astype(str)
df_RFM["Summary"] = 'A ' + df_RFM['definition R'].astype(str) +  ', ' + df_RFM['definition F'].astype(str) + ', ' +  df_RFM['definition M'].astype(str)+ ', ' + df_RFM['definition L'].astype(str) 

In [ ]:
df_RFM.drop(['Frequency','amount','lifetime','Avg_frequency','Avg_amount','definition F','definition R','definition M','definition L'], axis=1, inplace=True)
df_RFM.head()

,account_id,Rvalue,Active Period,Fvalue,Mvalue,Lvalue,RFML,Summary
0,799,1,2.9,1,1,3,1113,"A churned, low activity, low value, Long time ..."
1,1720,1,1.3,1,1,2,1112,"A churned, low activity, low value, Mid-life C..."
2,2029,1,3.5,1,1,3,1113,"A churned, low activity, low value, Long time ..."
3,459,1,4.1,1,1,3,1113,"A churned, low activity, low value, Long time ..."
4,2180,1,3.4,1,1,3,1113,"A churned, low activity, low value, Long time ..."


In [ ]:
df_RFM.to_csv('RFM.csv', index = False, encoding='utf-8')